In this notebook you will find feature engineering and modeling.
I have kept enough of the cleaning code to be followed but if you'd like the step by step process or to see my EDA please refer to 1_cleaning_and_EDA.ipynb


#### Feature Engineering
* Split Test and Train
* Define New Feature
    - Count Features
    - Feature by Feature
    - Total Admission Fees
    - Traveled Feature
* Add Features to Datasets

#### Modeling
* Encode Labels
* One Hot Encode
* Random Forest
* 

In [1]:
# Importing basics libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


In [2]:
# Importing data
stay_df = pd.read_csv('data/train_data.csv')
stay_test_df = pd.read_csv('data/test_data.csv')
stay_df.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [3]:
print(stay_df.shape, stay_test_df.shape)
stay_df.Stay.unique()

(318438, 18) (137057, 17)


array(['0-10', '41-50', '31-40', '11-20', '51-60', '21-30', '71-80',
       'More than 100 Days', '81-90', '61-70', '91-100'], dtype=object)

In [4]:
stay_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            318438 non-null  int64  
 1   Hospital_code                      318438 non-null  int64  
 2   Hospital_type_code                 318438 non-null  object 
 3   City_Code_Hospital                 318438 non-null  int64  
 4   Hospital_region_code               318438 non-null  object 
 5   Available Extra Rooms in Hospital  318438 non-null  int64  
 6   Department                         318438 non-null  object 
 7   Ward_Type                          318438 non-null  object 
 8   Ward_Facility_Code                 318438 non-null  object 
 9   Bed Grade                          318325 non-null  float64
 10  patientid                          318438 non-null  int64  
 11  City_Code_Patient                  3139

In [5]:
stay_df.describe()

,case_id,Hospital_code,City_Code_Hospital,Available Extra Rooms in Hospital,Bed Grade,patientid,City_Code_Patient,Visitors with Patient,Admission_Deposit
count,318438.000000,318438.000000,318438.000000,318438.000000,318325.000000,318438.000000,313906.000000,318438.000000,318438.000000
mean,159219.500000,18.318841,4.771717,3.197627,2.625807,65747.579472,7.251859,3.284099,4880.749392
std,91925.276847,8.633755,3.102535,1.168171,0.873146,37979.936440,4.745266,1.764061,1086.776254
min,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1800.000000
25%,79610.250000,11.000000,2.000000,2.000000,2.000000,32847.000000,4.000000,2.000000,4186.000000
50%,159219.500000,19.000000,5.000000,3.000000,3.000000,65724.500000,8.000000,3.000000,4741.000000
75%,238828.750000,26.000000,7.000000,4.000000,3.000000,98470.000000,8.000000,4.000000,5409.000000
max,318438.000000,32.000000,13.000000,24.000000,4.000000,131624.000000,38.000000,32.000000,11008.000000


In [6]:
stay_df.isna().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                             113
patientid                               0
City_Code_Patient                    4532
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
Stay                                    0
dtype: int64

In [7]:
stay_test_df.isna().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                              35
patientid                               0
City_Code_Patient                    2157
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
dtype: int64

In [8]:
# clean up column names first by replacing spaes with underscore

[stay_df.rename(columns={name: name.replace(" ", "_")}, inplace= True) for name in stay_df.columns if ' ' in name]

[stay_test_df.rename(columns={name: name.replace(" ", "_")}, inplace = True) for name in stay_test_df.columns if ' ' in name]

stay_df.columns, stay_test_df.columns

(Index(['case_id', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
        'Hospital_region_code', 'Available_Extra_Rooms_in_Hospital',
        'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed_Grade',
        'patientid', 'City_Code_Patient', 'Type_of_Admission',
        'Severity_of_Illness', 'Visitors_with_Patient', 'Age',
        'Admission_Deposit', 'Stay'],
       dtype='object'),
 Index(['case_id', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
        'Hospital_region_code', 'Available_Extra_Rooms_in_Hospital',
        'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed_Grade',
        'patientid', 'City_Code_Patient', 'Type_of_Admission',
        'Severity_of_Illness', 'Visitors_with_Patient', 'Age',
        'Admission_Deposit'],
       dtype='object'))

In [9]:
# fill missing Bed_Grade

stay_df.Bed_Grade.fillna(stay_df.Bed_Grade.mode()[0], inplace=True)

stay_test_df.Bed_Grade.fillna(stay_df.Bed_Grade.mode()[0], inplace=True)

stay_df.Bed_Grade = stay_df.Bed_Grade.astype('int')
stay_test_df.Bed_Grade = stay_test_df.Bed_Grade.astype('int')

In [10]:
# fill missing City_Code_Patient with the City_Code_Hospital of the sample

stay_df.loc[stay_df.City_Code_Patient.isnull(), 'City_Code_Patient'] = stay_df.City_Code_Hospital.loc[stay_df.City_Code_Patient.isnull()]

stay_test_df.loc[stay_test_df.City_Code_Patient.isnull(), 'City_Code_Patient'] = stay_test_df.City_Code_Hospital.loc[stay_test_df.City_Code_Patient.isnull()]

stay_df.City_Code_Patient = stay_df.City_Code_Patient.astype('int')
stay_test_df.City_Code_Patient = stay_test_df.City_Code_Patient.astype('int')

In [11]:
stay_df.case_id.nunique() == len(stay_df)

True

In [12]:
original_columns = stay_df.columns
original_columns

Index(['case_id', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available_Extra_Rooms_in_Hospital',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed_Grade',
       'patientid', 'City_Code_Patient', 'Type_of_Admission',
       'Severity_of_Illness', 'Visitors_with_Patient', 'Age',
       'Admission_Deposit', 'Stay'],
      dtype='object')

### Split Data

Splitting data first to ensure our feature engineering doesn't cause any leakage. 

In [13]:
X = stay_df.drop(['Stay', 'case_id'], axis=1)
y = stay_df['Stay']



X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=86)

final_case_ids = stay_test_df['case_id']
stay_test_df = stay_test_df.drop('case_id', axis=1)

In [14]:
y_test.shape, X_test.shape

((79610,), (79610, 16))

## Feature engineering

Create a feature counts by hospital of: 'patientid', 'Visitors_with_Patient', 'Age', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code'


for each of: 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code'
create a feature for count of:  'Available_Extra_Rooms_in_Hospital', 'Bed_Grade','Severity_of_Illness', 'Visitors_with_Patient'


Create total_admissions_cost for 'patienid'

### Count features

impute a count of how many samples have that feature in the .


In [15]:
features_tocount = ['Visitors_with_Patient', 'Age', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code'] 

def feature_totals(features, df):
    """""Takes a Dataframe and a list of features for each feature 
    it adds a count the features occurence in the train set to the df"""
    for feature in features:
        num_col = len(df.columns)
        total = X_train.groupby(feature).size().to_dict()
        df['_'.join(['Total', feature, 'Count'])] = df[feature].map(total)
        df['_'.join(['Total', feature, 'Count'])].fillna(0, inplace=True)
        assert len(df.columns) == (num_col + 1), 'column did not add'
    return df


### feature by feature

impute counts of occurrences of a feature by another feature
count of rooms available, bed grade or visitors by location, type or department of the hospital

In [16]:
hosp_descriptor = ['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code']

hosp_features = ['Available_Extra_Rooms_in_Hospital', 'Bed_Grade', 'Visitors_with_Patient']

In [17]:
def two_feature_countby(to_count, by_name, df):
    """"Takes two lists of features and a dataframe, adds new feature to 
    dataframe that is the count of one feature by the other"""
    
    for count_of in to_count:
        for count_by in by_name:
            num_col = len(df.columns)
            count = X_train.groupby([count_of, count_by]).size().reset_index()
            count.columns = [count_of, count_by, '_'.join([count_by, 'by', count_of ])]
            
            df = df.merge(count, how='left', on=[count_of, count_by])
            df['_'.join([count_by, 'by', count_of ])].fillna(0, inplace=True)
            assert len(df.columns) == (num_col + 1), 'column did not add'
            
    return df

### patient traveled

if the patient traveled to get to the hospital it may increase the time needed to recover enough to get home. 

create a binary feature with one representing that the patient lives in a different region then the hospital 

In [18]:
# need city_code_hospital values split into list by reagion
# if patient 'City_Code_Patient' not in region_list that matches Hospital_region_code
#  add 1 for not in, else leave 0

z_region = X_train[X_train.Hospital_region_code == 'Z']['City_Code_Hospital'].unique()
x_region = X_train[X_train.Hospital_region_code == 'X']['City_Code_Hospital'].unique()
y_region = X_train[X_train.Hospital_region_code == 'Y']['City_Code_Hospital'].unique()


def define_region_list(hosp_region):
    """""takes in a samples region and returns region list of city codes in that region"""
    if hosp_region == 'Z':
        return z_region
    elif hosp_region == 'X':
        return x_region
    elif hosp_region == 'Y':
        return y_region
    assert  hosp_region not in ['Z', 'X', 'Y'], 'reagion not machted'
    



def patiend_travel(df):
    """"Takes in a Dataframe, returns list of 1 and 0s for  whether the patient_traveled or not"""
    traveled = []
    locations = zip(df['Hospital_region_code'], df['City_Code_Patient'])
    for hosp_region, patient_home in locations:
        
        region_list = define_region_list(hosp_region)
        
        if patient_home not in region_list:
            traveled.append(1)
        elif patient_home in region_list:
            traveled.append(0)
    return traveled

### Apply functions to Dataframes 

In [19]:
def add_features(df):
    samples = len(df)
    
    df = feature_totals(features_tocount, df)
    assert len(df) == samples, 'feature_totals missing samples'
    
    df = two_feature_countby(hosp_descriptor, hosp_features, df)
    #assert len(df) == samples, 'two_feature_countby missing samples'
    
    df['Patient_traveled'] = patiend_travel(df)
    #assert len(df) == samples, 'patiend_travel missing samples'
    
    return df

In [20]:
X_train = add_features( X_train)
X_test = add_features( X_test)
final_X_test = add_features(stay_test_df)

<ipython-input-15-978cb138c8c9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_'.join(['Total', feature, 'Count'])] = df[feature].map(total)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [21]:
new_features = set(X_train.columns) - set(original_columns )
len(new_features)

31

In [22]:
X_train[new_features]

,Visitors_with_Patient_by_Hospital_region_code,Total_Ward_Type_Count,Total_Hospital_type_code_Count,Total_Visitors_with_Patient_Count,Available_Extra_Rooms_in_Hospital_by_City_Code_Hospital,Bed_Grade_by_Hospital_region_code,Bed_Grade_by_City_Code_Hospital,Visitors_with_Patient_by_Hospital_type_code,Available_Extra_Rooms_in_Hospital_by_Hospital_region_code,Total_Hospital_code_Count,...,Visitors_with_Patient_by_City_Code_Hospital,Total_City_Code_Hospital_Count,Total_Age_Count,Available_Extra_Rooms_in_Hospital_by_Ward_Facility_Code,Available_Extra_Rooms_in_Hospital_by_Department,Visitors_with_Patient_by_Ward_Type,Total_Ward_Facility_Code_Count,Available_Extra_Rooms_in_Hospital_by_Ward_Type,Bed_Grade_by_Ward_Type,Bed_Grade_by_Ward_Facility_Code
0,2401,95810,51778,6953,10418,38786,15199,1628,25874,24804,...,1036,38869,26867,10418,55358,3794,38869,34668,36989,15199
1,38930,79840,51778,103651,12002,29669,13208,22502,25258,24804,...,16746,38869,36259,12002,57491,25667,38869,33259,27507,13208
2,2575,95810,34477,14153,6679,15727,7969,1822,13315,5350,...,1169,23661,47844,5893,55358,6620,20929,34668,34203,7161
3,21461,58251,34477,103651,460,9461,4475,15990,868,6140,...,11179,23661,5856,441,528,29789,20929,2254,10100,3971
4,38930,58251,51778,103651,10418,13646,7411,22502,25874,24804,...,16746,38869,47844,10418,5625,29789,38869,15247,10100,7411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238823,2401,79840,107489,6953,7630,29669,6981,3257,25874,16023,...,670,26702,47844,7630,55358,1529,26702,17962,27507,6981
238824,323,79840,51778,878,12002,13646,7411,177,25258,24804,...,137,38869,30713,12002,57491,301,38869,33259,14686,7411
238825,38930,79840,107489,103651,4361,38786,8068,44089,25258,8435,...,9706,19619,47733,6014,57491,25667,26322,33259,30434,11016
238826,43260,79840,107489,103651,13380,37667,16152,44089,29715,13014,...,18526,41431,47733,13380,10686,25667,41431,12880,27507,16152


In [23]:
X_train.shape, X_test.shape, final_X_test.shape

((238828, 47), (79610, 47), (137057, 47))

In [24]:
X_train.select_dtypes(exclude=[np.number]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238828 entries, 0 to 238827
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Hospital_type_code    238828 non-null  object
 1   Hospital_region_code  238828 non-null  object
 2   Department            238828 non-null  object
 3   Ward_Type             238828 non-null  object
 4   Ward_Facility_Code    238828 non-null  object
 5   Type_of_Admission     238828 non-null  object
 6   Severity_of_Illness   238828 non-null  object
 7   Age                   238828 non-null  object
dtypes: object(8)
memory usage: 16.4+ MB


In [25]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238828 entries, 0 to 238827
Data columns (total 47 columns):
 #   Column                                                     Non-Null Count   Dtype  
---  ------                                                     --------------   -----  
 0   Hospital_code                                              238828 non-null  int64  
 1   Hospital_type_code                                         238828 non-null  object 
 2   City_Code_Hospital                                         238828 non-null  int64  
 3   Hospital_region_code                                       238828 non-null  object 
 4   Available_Extra_Rooms_in_Hospital                          238828 non-null  int64  
 5   Department                                                 238828 non-null  object 
 6   Ward_Type                                                  238828 non-null  object 
 7   Ward_Facility_Code                                         238828 non-null  object 

### One hot encode

In [26]:
X_train = pd.get_dummies(X_train, prefix=['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type_of_Admission', 'Severity_of_Illness', 'Age'])
X_test = pd.get_dummies(X_test, prefix=['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type_of_Admission', 'Severity_of_Illness', 'Age'])
final_X_test = pd.get_dummies(final_X_test, prefix=['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type_of_Admission', 'Severity_of_Illness', 'Age'])

### Preprocessing and Modeling

Random Forest requires the least preprocessing. Independent features are used for decision boundaries so scaling shouldn't affect it .

The target variable does need to be encoded. 


In [27]:
label = LabelEncoder()
label.fit(y_train)
label.transform(y_train)
label.transform(y_test)
y_test.shape, X_test.shape

((79610,), (79610, 82))

In [28]:
X_train.shape, y_train.shape

((238828, 82), (238828,))

In [29]:
#pd.options.display.max_rows = 100


In [30]:
RF = RandomForestClassifier(random_state=86)
#pipeline = Pipeline(steps = [
 #              ('preprocessor', preprocessor)
  #            ,('regressor',RandomForestClassifier())
   #        ])

RF.fit(X_train, y_train)
rf_pred = RF.predict(X_test)


In [31]:
print(classification_report(y_test, rf_pred))

                    precision    recall  f1-score   support

              0-10       0.29      0.17      0.22      5912
             11-20       0.38      0.44      0.41     19349
             21-30       0.42      0.53      0.47     22041
             31-40       0.33      0.27      0.30     13734
             41-50       0.08      0.02      0.04      2933
             51-60       0.41      0.45      0.43      8752
             61-70       0.15      0.03      0.05       721
             71-80       0.29      0.12      0.17      2526
             81-90       0.41      0.29      0.34      1220
            91-100       0.38      0.10      0.16       719
More than 100 Days       0.56      0.49      0.52      1703

          accuracy                           0.38     79610
         macro avg       0.34      0.27      0.28     79610
      weighted avg       0.37      0.38      0.37     79610



In [32]:
RF.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 86,
 'verbose': 0,
 'warm_start': False}

In [35]:
RFC_params = {'n_estimators': [200, 400, 600, 800, 1000, 1200],
              'max_features': ['auto', 'log2'],
              'max_depth': [10, 20, 30, 40, 50, 60, 70,  None],
              'min_samples_split': [2, 5, 10, 15, 20],
              'min_samples_leaf': [1, 2, 5, 10, 15]
             }
RF = RandomForestClassifier(random_state=86)
RFGS = RandomizedSearchCV(RF, RFC_params, random_state=86)

In [34]:
RFGS.fit(X_train, y_train)
RFGS.best_params_

/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'n_jobs': 2,
 'n_estimators': 1000,
 'min_samples_split': 5,
 'min_samples_leaf': 10,
 'max_features': 'auto',
 'max_depth': 40}

In [38]:
RF2 = RandomForestClassifier(random_state=86, n_jobs= 2, n_estimators = 1000, 
                             min_samples_split = 5, min_samples_leaf = 10, 
                             max_features = 'auto', max_depth= 40)

RF2.fit(X_train, y_train)
rf_pred2 = RF2.predict(X_test)
print(classification_report(y_test, rf_pred2))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

              0-10       0.45      0.11      0.18      5912
             11-20       0.41      0.50      0.45     19349
             21-30       0.42      0.66      0.52     22041
             31-40       0.42      0.24      0.30     13734
             41-50       0.00      0.00      0.00      2933
             51-60       0.41      0.50      0.45      8752
             61-70       0.00      0.00      0.00       721
             71-80       0.43      0.01      0.03      2526
             81-90       0.41      0.22      0.28      1220
            91-100       0.36      0.01      0.01       719
More than 100 Days       0.54      0.45      0.49      1703

          accuracy                           0.42     79610
         macro avg       0.35      0.25      0.25     79610
      weighted avg       0.40      0.42      0.38     79610



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
rf_pred2_final = RF2.predict(final_X_test)



In [42]:
rf_pred2_final

array(['0-10', '51-60', '21-30', ..., '21-30', '11-20', '51-60'],
      dtype=object)

In [46]:
rf_submission = pd.DataFrame({'case_id': final_case_ids, 'Stay': rf_pred2_final})


rf_submission

,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60
...,...,...
137052,455491,21-30
137053,455492,0-10
137054,455493,21-30
137055,455494,11-20


In [48]:
#rf_submission.to_csv('data/rf_submission.csv', index =False)

This submission receives public accuracy score of: 42.0013863047682
and a private accuracy score of: 41.7048701890922
if the contest was still going it would be in the top 300 submissions.